<a href="https://colab.research.google.com/github/lmkwytnicholas/nic.github.io/blob/master/contentBasedRecommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data Access & EDA
* Preprocessing, cleaning part of Data Access

In [ ]:
# Movie Content Based Reocmmendation System
import pandas as pd
import numpy as np

In [ ]:
credits = pd.read_csv('/content/drive/MyDrive/Tech I.S./00 Datasets/tmdb_5000_credits.csv')
movies_df = pd.read_csv('/content/drive/MyDrive/Tech I.S./00 Datasets/tmdb_5000_movies.csv')

In [ ]:
credits.head(2)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [ ]:
movies_df.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500


In [ ]:
print('credits df shape:',credits.shape)
print('movies df shape:',movies_df.shape)

credits df shape: (4803, 4)
movies df shape: (4803, 20)


In [ ]:
# Merge credits with movies df
credits_column_renamed = credits.rename(index=str,columns = {'movie_id':'id'})
movies_df_merge = movies_df.merge(credits_column_renamed,on='id')
movies_df_merge.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title_x,vote_average,vote_count,title_y,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [ ]:
# Drop unnecessary features
movies_cleaned_df = movies_df_merge.drop(columns=['homepage','title_x','title_y','status','production_countries'])
movies_cleaned_df.head(2)

,budget,genres,id,keywords,original_language,original_title,overview,popularity,production_companies,release_date,revenue,runtime,spoken_languages,tagline,vote_average,vote_count,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Enter the World of Pandora.,7.2,11800,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]","At the end of the world, the adventure begins.",6.9,4500,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."


In [ ]:
movies_cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4803 entries, 0 to 4802
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   id                    4803 non-null   int64  
 3   keywords              4803 non-null   object 
 4   original_language     4803 non-null   object 
 5   original_title        4803 non-null   object 
 6   overview              4800 non-null   object 
 7   popularity            4803 non-null   float64
 8   production_companies  4803 non-null   object 
 9   release_date          4802 non-null   object 
 10  revenue               4803 non-null   int64  
 11  runtime               4801 non-null   float64
 12  spoken_languages      4803 non-null   object 
 13  tagline               3959 non-null   object 
 14  vote_average          4803 non-null   float64
 15  vote_count           

In [ ]:
# Content based recommendations will be operationalized as relating to movie's plot summaries 
# Example:
movies_cleaned_df['overview'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'

# TF-IDF Vectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfv = TfidfVectorizer(min_df=3,max_features=None,
                      strip_accents='unicode',
                      analyzer='word',
                      token_pattern=r'\w{1,}',
                      ngram_range=(1,3),
                      stop_words='english')

# Fill NaNs with empty string
movies_cleaned_df['overview'] = movies_cleaned_df['overview'].fillna('')

# Fit the TFIDF on 'overview' text
tfv_matrix = tfv.fit_transform(movies_cleaned_df['overview'])
tfv_matrix

<4803x10417 sparse matrix of type '<class 'numpy.float64'>'
	with 127220 stored elements in Compressed Sparse Row format>

In [ ]:
print(tfv_matrix.shape)
print('-'*100)
tfv_matrix.toarray()

(4803, 10417)
----------------------------------------------------------------------------------------------------


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
# Cosine Similarity
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(tfv_matrix,tfv_matrix)
cosine_sim

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.02405178, 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.0173424 , 0.        ,
        0.00981313],
       ...,
       [0.        , 0.02405178, 0.0173424 , ..., 1.        , 0.01672279,
        0.0141502 ],
       [0.        , 0.        , 0.        , ..., 0.01672279, 1.        ,
        0.01137347],
       [0.        , 0.        , 0.00981313, ..., 0.0141502 , 0.01137347,
        1.        ]])

In [ ]:
# Indices of movie titles
indices = pd.Series(movies_cleaned_df.index, index=movies_cleaned_df['original_title']).drop_duplicates()
indices[:20]

original_title
Avatar                                          0
Pirates of the Caribbean: At World's End        1
Spectre                                         2
The Dark Knight Rises                           3
John Carter                                     4
Spider-Man 3                                    5
Tangled                                         6
Avengers: Age of Ultron                         7
Harry Potter and the Half-Blood Prince          8
Batman v Superman: Dawn of Justice              9
Superman Returns                               10
Quantum of Solace                              11
Pirates of the Caribbean: Dead Man's Chest     12
The Lone Ranger                                13
Man of Steel                                   14
The Chronicles of Narnia: Prince Caspian       15
The Avengers                                   16
Pirates of the Caribbean: On Stranger Tides    17
Men in Black 3                                 18
The Hobbit: The Battle of the Five 

#Function - Find Top 10 Recommended Movies

In [ ]:
def give_rec(title,sim=cosine_sim):
  # Get the index corresponding to original_title
  idx=indices[title]

  # Get the pairwise similarity score
  sim_scores = list(enumerate(cosine_sim[idx]))

  # Sort the movies
  sim_scores = sorted(sim_scores,key=lambda x:x[1],reverse=True)

  # Scores of the 10 most similar movies
  sim_scores = sim_scores[1:11]

  # Movie indices
  movie_indices = [i[0] for i in sim_scores]

  # Top 10 most similar movies
  return movies_cleaned_df['original_title'].iloc[movie_indices]

# Test Recommendation System

In [ ]:
print('Search Recommended Movies')
x=input()
give_rec(x)

Search Recommended Movies
The Matrix


2088                                  Pulse
0                                    Avatar
1341                       Obitaemyy Ostrov
775                               Supernova
4395                           The Specials
1281                                Hackers
2996                               Commando
354         The Girl with the Dragon Tattoo
2992                                Oliver!
100     The Curious Case of Benjamin Button
Name: original_title, dtype: object